<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>
    
### Ονοματεπώνυμο: Καμπουγέρης Χαράλαμπος
### ΑΜ: 03120098

<html><font size=4><B>1. Κατασκευή γράφου και υπολογισμός μετρικών</B></font>
    <li><B>1(i).</B> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/maayan-pdzbase/"> KONECT (PDZBase) </a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$. Οι κόμβοι του γράφου αναπαριστούν πρωτεΐνες και οι ακμές την αλληλεπίδραση αυτών. Στην σελίδα <a href="http://konect.cc/networks/maayan-pdzbase/"> KONECT (PDZBase)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a><BR>
        <B><U>Υπόδειξη:</U></B> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί.<BR></html>

In [1]:
import matplotlib
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [2]:
duplicate_edges = 0
duplicate_edges_rev = 0

graph = nx.Graph()

with open("out.maayan-pdzbase") as file:
    for l in file:
        u_str, v_str = l.split('\t')
        u, v = int(u_str) - 1, int(v_str) - 1
        duplicate_edges += graph.has_edge(u, v)
        duplicate_edges_rev += graph.has_edge(v, u)
        graph.add_edge(u, v)

In [3]:
print(graph)
print('Duplicate Edges:', duplicate_edges)
print('Duplicate Edges (reversed):', duplicate_edges_rev)

Graph with 212 nodes and 244 edges
Duplicate Edges: 0
Duplicate Edges (reversed): 0


 <li><B>1(ii).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li><BR>

In [4]:
# Number of nodes
initial_node_count = graph.number_of_nodes()
print("Number of Nodes:", initial_node_count)

# Number of edges
initial_edge_count = graph.number_of_edges()
print("Number of Edges:", initial_edge_count)

# Average degree of nodes
average_node_degree = sum(dict(graph.degree()).values()) / initial_node_count
print("Average Degree of Nodes:", average_node_degree)

# Number of connected components
initial_connected_components = nx.number_connected_components(graph)
print("Number of Connected Components:", initial_connected_components)

Number of Nodes: 212
Number of Edges: 244
Average Degree of Nodes: 2.30188679245283
Number of Connected Components: 19


<li><B>2 Προεργασία για την πρόβλεψη συνδέσμων. 2(i). </B>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li><BR>

In [5]:
adj_matrix = nx.adjacency_matrix(graph).todense()

# Calculate the set of unconnected node pairs
unconnected_pairs = []
num_nodes = adj_matrix.shape[0]

for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        if adj_matrix[i, j] == 0:
            unconnected_pairs.append((i, j))

# Number of unconnected pairs
num_unconnected_pairs = len(unconnected_pairs)
print("Number of Unconnected Pairs:", num_unconnected_pairs)

Number of Unconnected Pairs: 22124


<li><B>2(ii).</B> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li><BR>

In [6]:
removable_edges = []
copy_graph = graph.copy()
for edge in graph.edges():
    copy_graph.remove_edge(*edge)
    copy_node_cnt = copy_graph.number_of_nodes()
    copy_conn_comps = nx.number_connected_components(copy_graph)
    # remove edges that are not self edges && when deleted connected comps and node cnt stay the same
    if (copy_conn_comps == initial_connected_components and copy_node_cnt == initial_node_count and edge[0] != edge[1]):
        removable_edges.append(edge)
    else:
        copy_graph.add_edge(*edge)

print("Removable Edges:", len(removable_edges))

Removable Edges: 49


##### Παρατηρήσεις

* Χρησιμοποιήθηκε ένα copy του γράφου ώστε να μπορούμε να κάνουμε τις απαραίτητες αλλαγές κάθε φορά και να ελέγχουμε αν η αποκοπή της ακμής αυτής σπάει κάποια από τις προϋποθέσεις
* Το self edge δεν προστέθηκε στο σύνολο των removable edges

<li><B>2(iii).</B> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 2(i) και 2(ii) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable). <BR>
    <li><B>2(iv).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 2(ii). <BR>
    <li><B>2(v).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li><BR>

In [7]:
df_uncon_pairs = pd.DataFrame(unconnected_pairs)
zeros = [0] * len(df_uncon_pairs)
df_uncon_pairs.insert(2, 'link', zeros)
print(df_uncon_pairs)

         0    1  link
0        0    3     0
1        0    4     0
2        0    5     0
3        0    6     0
4        0    7     0
...    ...  ...   ...
22119  207  211     0
22120  208  210     0
22121  208  211     0
22122  209  210     0
22123  209  211     0

[22124 rows x 3 columns]


In [8]:
df_rem_edges = pd.DataFrame(removable_edges)
ones = [1] * len(df_rem_edges)
df_rem_edges.insert(2, 'link', ones)
print(df_rem_edges)

      0    1  link
0     1  113     1
1   113    7     1
2   113  128     1
3   113  140     1
4   113  143     1
5     2   56     1
6     4  111     1
7     4   22     1
8    22  178     1
9    22   31     1
10   44   13     1
11   44   18     1
12   44   36     1
13    6   20     1
14   20   13     1
15   20   18     1
16   20   36     1
17    7   18     1
18   18   14     1
19   18   15     1
20   18   17     1
21   18  100     1
22   18  105     1
23   18   19     1
24   70   17     1
25   70  145     1
26    8   10     1
27    8  115     1
28  115  114     1
29   13  105     1
30   13   15     1
31   13   17     1
32   13  189     1
33   13   19     1
34   13   42     1
35   13   61     1
36   13   83     1
37   12   60     1
38   19  186     1
39   19   59     1
40  178  133     1
41  143  147     1
42   25   37     1
43   33  126     1
44   33  145     1
45   33  158     1
46  158   46     1
47   39  146     1
48  146  123     1


In [9]:
df_uncon_pairs.sort_index(inplace=True)
df_rem_edges.sort_index(inplace=True)

In [10]:
frames = [df_rem_edges, df_uncon_pairs]

G_merged = pd.concat(frames)

In [11]:
G_merged

,0,1,link
0,1,113,1
1,113,7,1
2,113,128,1
3,113,140,1
4,113,143,1
...,...,...,...
22119,207,211,0
22120,208,210,0
22121,208,211,0
22122,209,210,0


<li><B>2(iv).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 2(ii). <BR>

In [12]:
#build graph G' = G_reduced
G_reduced = graph.copy()
G_reduced.remove_edges_from(removable_edges)

In [13]:
# Number of nodes
node_count = G_reduced.number_of_nodes()
print("Number of Nodes:", node_count)

# Number of edges
edge_count = G_reduced.number_of_edges()
print("Number of Edges:", edge_count)

# Average degree of nodes
average_node_degree = sum(dict(G_reduced.degree()).values()) / node_count
print("Average Degree of Nodes:", average_node_degree)

# Number of connected components
connected_components = nx.number_connected_components(G_reduced)
print("Number of Connected Components:", connected_components)

Number of Nodes: 212
Number of Edges: 195
Average Degree of Nodes: 1.8396226415094339
Number of Connected Components: 19


Βλέπουμε ότι όπως περιμέναμε, ο αριθμός των κόμβων και των συνεκτικών συνιστωσών παραμένει σταθερός ενώ ο αριθμός των ακμών και ο μέσος βαθμός κόμβου μειώθηκε, κάτι το οποίο είναι λογικό αφού αφαιρέθηκαν ακμές από τον γράφο.

<html><font size=4><B>3. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </B></font><BR><BR>
    <li><B>3(i).</B> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <B>Node2vec</B>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <BR>Για την επιλογή των παραμέτρων: 
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

Αρχικά κάνουμε rename στις στήλες προκειμένου να μπορέσουμε να χρησιμοποίησουμε την συνάρτηση `nx.from_pandas_edgelist()`.

In [14]:
G_merged = G_merged.rename(columns={0: "source", 1: "target"})

In [15]:
G_merged_graph = nx.from_pandas_edgelist(G_merged)

In [18]:
from node2vec import Node2Vec
import multiprocessing

#generate walks
node2vec_walker = Node2Vec(
    G_reduced, 
    p = 1.0, 
    q = 1.0, 
    dimensions = 128, 
    num_walks = 10, 
    walk_length = 80, 
    workers = multiprocessing.cpu_count()
)

#train node2vec model
n2v_model = node2vec_walker.fit(window = 10, min_count = 1)

Generating walks (CPU: 7): 100%|██████████| 1/1 [00:00<00:00, 34.35it/s]


<html><font>
    <li><B>3(ii).</B> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><BR>
    <li><B>3(iii).</B> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><BR></font></html>

In [19]:
#apply trained Node2Vec model on each node pair of the dataframe "unconnected+removables"
#uncomment:
x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(G_merged['source'], G_merged['target'])]

Επειδή θέλουμε numpy arrays ενώ παραπάνω τα χαρακτηριστικά τα έχουμε σε μορφή list κάνουμε την μετατροπή με την συνάρτηση `np.array()`. Τα χαρακτηριστικά μας είναι τα x που υπολογίσαμε παραπάνω ενώ τα labels μας είναι η στήλη link.

In [20]:
x = np.array(x)  
y = G_merged['link']

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 0)  #uncomment and complete

In [22]:
#uncomment whole cell
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(max_depth=2, random_state=0) 
classifier.fit(X_train, y_train)   #uncomment and complete

RandomForestClassifier(max_depth=2, random_state=0)

In [23]:
predictions = classifier.predict(X_test)
accuracy = accuracy_score(predictions, y_test)

In [24]:
print("The accuracy of the model is", accuracy)

The accuracy of the model is 0.9976551226551227
